In [2]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

class StockTrader:
    def __init__(self, alpaca_api_key, alpaca_secret_key, base_url='https://paper-api.alpaca.markets'):
        # Initialize Alpaca API
        self.api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url, api_version='v2')
        self.model = None
        self.test_mode = False

    def fetch_historical_data(self, symbol, start_date, end_date):
        # Fetch historical daily bars
        bars = self.api.get_bars(
            symbol,
            '1Day',
            start=start_date.isoformat(),
            end=end_date.isoformat()
        ).df
        bars = bars[bars['symbol'] == symbol]
        return bars

    def prepare_features(self, data):
        # Prepare features for the model
        data['return'] = data['close'].pct_change()
        data['volatility'] = data['close'].rolling(window=5).std()
        data['momentum'] = data['close'] / data['close'].shift(5) - 1
        data['ma5'] = data['close'].rolling(window=5).mean()
        data['ma10'] = data['close'].rolling(window=10).mean()
        data['ma_ratio'] = data['ma5'] / data['ma10']
        data = data.dropna()
        features = data[['return', 'volatility', 'momentum', 'ma_ratio']]
        return features

    def train_model(self, symbol, start_date, end_date):
        # Fetch and prepare data
        data = self.fetch_historical_data(symbol, start_date, end_date)
        features = self.prepare_features(data)
        target = data['close'].shift(-1).dropna()
        features = features.iloc[:-1]  # Align features and target

        # Split into training and testing sets
        split_index = int(len(features) * 0.8)
        X_train, X_test = features[:split_index], features[split_index:]
        y_train, y_test = target[:split_index], target[split_index:]

        # Train the model
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        print(f"Model trained. Test MSE: {mse:.4f}")

        if self.test_mode:
            self.backtest_strategy(X_test, y_test, y_pred)

    def backtest_strategy(self, X_test, y_test, y_pred):
        # Simulate trading strategy based on predictions
        initial_capital = 100000
        positions = pd.DataFrame(index=X_test.index).fillna(0.0)
        positions['signal'] = np.where(y_pred > X_test['return'], 1, -1)
        positions['holdings'] = positions['signal'] * y_test

        portfolio = positions['holdings'].cumsum() * 100  # Assuming 100 shares per trade
        portfolio += initial_capital

        # Plot the portfolio value over time
        plt.figure(figsize=(10, 5))
        plt.plot(portfolio)
        plt.title('Backtested Portfolio Value')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value ($)')
        plt.show()

        total_return = (portfolio.iloc[-1] - initial_capital) / initial_capital * 100
        print(f"Total return from backtesting: {total_return:.2f}%")

    def predict(self, symbol):
        # Predict the next day's closing price
        end_date = datetime.now()
        start_date = end_date - timedelta(days=60)  # Get recent data for features
        data = self.fetch_historical_data(symbol, start_date, end_date)
        features = self.prepare_features(data)

        if features.empty:
            print("Not enough data to make a prediction.")
            return

        latest_features = features.iloc[-1].values.reshape(1, -1)
        predicted_price = self.model.predict(latest_features)[0]
        current_price = data['close'].iloc[-1]
        expected_return = (predicted_price - current_price) / current_price * 100

        print(f"Predicted next closing price for {symbol}: ${predicted_price:.2f}")
        print(f"Expected return: {expected_return:.2f}%")

        return expected_return

    def execute_trade(self, symbol, expected_return, risk_tolerance=0.05):
        # Decide whether to buy or sell based on expected return
        if expected_return > risk_tolerance:
            # Place a buy order
            try:
                self.api.submit_order(
                    symbol=symbol,
                    qty=10,  # Number of shares to buy
                    side='buy',
                    type='market',
                    time_in_force='day'
                )
                print(f"Placed a market buy order for {symbol}")
            except Exception as e:
                print(f"Error placing buy order: {e}")
        elif expected_return < -risk_tolerance:
            # Place a sell order
            try:
                self.api.submit_order(
                    symbol=symbol,
                    qty=10,  # Number of shares to sell
                    side='sell',
                    type='market',
                    time_in_force='day'
                )
                print(f"Placed a market sell order for {symbol}")
            except Exception as e:
                print(f"Error placing sell order: {e}")
        else:
            print(f"No trade executed for {symbol}. Expected return does not meet risk tolerance.")

    def run(self, symbol, test_mode=False):
        self.test_mode = test_mode
        start_date = datetime.now() - timedelta(days=365*2)
        end_date = datetime.now() - timedelta(days=1)

        # Train the model
        self.train_model(symbol, start_date, end_date)

        if not self.test_mode:
            # Predict and trade
            expected_return = self.predict(symbol)
            if expected_return is not None:
                self.execute_trade(symbol, expected_return)


In [3]:
if __name__ == '__main__':
    alpaca_api_key = 'PKJATKYKR4NWCYALKL58'
    alpaca_secret_key = '8aFRppO0HdabLAO8b05ZImUwSd0bEmv4hqpU0Hqs'

    trader = StockTrader(alpaca_api_key, alpaca_secret_key)
    # To backtest the model
    trader.run('AAPL', test_mode=True)
    # To predict and trade on current data
    trader.run('AAPL', test_mode=False)


APIError: Invalid format for parameter start: error parsing '2022-12-03T13:32:03.137224' as RFC3339 or 2006-01-02 time: parsing time "2022-12-03T13:32:03.137224": extra text: "T13:32:03.137224"